In [ ]:
from IPython.display import HTML
HTML(filename='/Users/hubatsl/Desktop/SPT/Us/SPT/Python/hide_code.html')

In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import sys
import tifffile
import time
# Add path to python modules
sys.path.append(os.getcwd()[0:-7]+'src') 
from MovieTracks import (DiffusionFitter, OffRateFitter, ParticleFinder)
# Plotting becomes part of the notebook
%matplotlib inline
# File definitions that are machine specific
xls_file = '/Users/hubatsl/Desktop/Florent/PAR3_flows/PAR3_flows.xlsx'
output_file_name = '/Users/hubatsl/Desktop/Florent/PAR3_flows/PAR3_flows.csv'

# Can we see PAR-3 being advected using SPT during establishment?

**Run thresholding on movie movieNo, read and write threshold from excel file**

In [ ]:
movieNo = 14
exec(open("calibrate_feature_finder.py").read())

**Read time stamps for each movie, write to excel file**

In [ ]:
%%capture
exec(open('../src/params_to_excel.py').read())

**Run single particle tracking on all movies specified in SPT.xlsx, use parameters specified in SPT.xlsx. Save trajectories frame for each movie in .csv file.**

In [ ]:
for i, path in enumerate(xls_db['Folder']):
    interval = xls_db['IntervalReal'][i]
    StartFrame = xls_db['StartFrame'][i]
    mTL = 40
    p = DiffusionFitter(path, xls_db['Threshold'][i],
                        minTrackLength=mTL, startFrame=StartFrame)
    p.analyze()
    p.append_output_to_csv(output_file_name, p.gData())
    p.trajectories.to_csv('/'.join(p.stackPath.split('/')[:-1])+'/'+p.stackName+'.csv')

**Read in the trajectories frame saved in .csv file for each movie.**

In [ ]:
%%capture
df_list=[]
for i, path in enumerate(xls_db['Folder']):
    interval = xls_db['IntervalReal'][i]
    StartFrame = xls_db['StartFrame'][i]
    mTL = 40
    p = DiffusionFitter(path, xls_db['Threshold'][i],
                        minTrackLength=mTL, startFrame=StartFrame)
    df_list.append(pd.read_csv('/'.join(p.stackPath.split('/')[:-1])+'/'+p.stackName+'.csv'))

**Compile list of tracks and their starting and ending x and y positions.**

In [ ]:
xdiff = []
ydiff = []
xmean = []
ymean = []
xerr = []
yerr = []
li_length = []
early = [0, 1, 3, 4, 6, 7, 9, 10, 12, 13] # Use this for early flows
late = [15, 16, 17, 18]
early_a_to_l = [0, 1, 6, 7, 12, 13]
early_a_to_r = [3, 4, 9, 10]
list_to_evaluate = early_a_to_l
for i, df in enumerate(df_list):
    if i in list_to_evaluate:
        li = [df.loc[df.particle==part] for part in df.particle.unique()] 
        # Filter out the posterior particles, 0 is P left, 1 is A left
        if xls_db['Direction'][i] == 0:
            li_ant = [ part for part in li if part.x.tolist()[0] <= xls_db['PixelCut'][i]]
        elif xls_db['Direction'][i] == 1:
            li_ant = [ part for part in li if part.x.tolist()[0] >= xls_db['PixelCut'][i]]
        # Get difference between start and end positions in x and y direction
        if xls_db['Direction'][i] == 0:
            li_xdiff = [-(part.x.tolist()[-1]-part.x.tolist()[0]) for part in li_ant]
        elif xls_db['Direction'][i] == 1:
            li_xdiff = [part.x.tolist()[-1]-part.x.tolist()[0] for part in li_ant]
        li_ydiff = [part.y.tolist()[-1]-part.y.tolist()[0] for part in li_ant]
        li_length.append([len(part.y.tolist()) for part in li_ant])
        xdiff.append(li_xdiff)
        ydiff.append(li_ydiff)
        xmean.append(np.mean(li_xdiff))
        ymean.append(np.mean(li_ydiff))
        xerr.append(np.std(li_xdiff)/len(li_xdiff)**0.5)
        yerr.append(np.std(li_ydiff)/len(li_ydiff)**0.5)
xdiff_flat = [item for sublist in xdiff for item in sublist]
ydiff_flat = [item for sublist in ydiff for item in sublist]

**Plot histogram of net movement of each particle in x and y directions.**

In [ ]:
plt.hist(ydiff_flat, 30)
plt.hist(xdiff_flat, 30)
plt.show()